In [2]:
import numpy as np
import pandas as pd
#import Evaluation.metric_evaluation as eval
#from Evaluation.Metrics import accuracy as ac
import time
import pickle
import argparse
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool, cpu_count

In [3]:
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)

def predict_function(test_session, pr, items_to_predict, cut_off=20, 
                     session_key='SessionId', item_key='ItemId', time_key='Time'):
    test_session.sort_values([time_key], inplace=True)
    # get first and only session_id (as we grouped it before calling this method)
    session_id = test_session[session_key].unique()[0]

    log_columns = ["session_id", "input_items", "input_count", "position", "remaining_items", "remaining_count", "predictions"]
    log_df = pd.DataFrame(columns = log_columns)

    session_length = len(test_session)
    for i in range(session_length -1):
        # use current item as reference point (rest is for testing)
        current_item_id = test_session[item_key].values[i]
        
        # make predictions
        preds = pr.predict_next(session_id, current_item_id, items_to_predict)
        # extract top-n predicted items
        preds[np.isnan(preds)] = 0
        preds.sort_values( ascending=False, inplace=True )
        
        topn_preds = preds[:cut_off]
        
        # log results
        current_input_set = test_session[item_key].values[:i+1]
        remaining_test_set = test_session[item_key].values[i+1:]
        position = "MID"
        if i == 0:
            position = "FIRST"
        if len(remaining_test_set) == 1:
            position = "LAST"
        
        # use np.array_str as the a new line charachter will be automatically set otherweise
        log_df = log_df.append({
            "session_id": session_id,
            "input_items":  ','.join(map(str, current_input_set)),
            "input_count":  len(current_input_set),
            "position": position,
            "remaining_items":  ','.join(map(str, remaining_test_set)),
            "remaining_count":  len(remaining_test_set),
            "predictions": ','.join(map(str, topn_preds.index.values))
      }, ignore_index=True)
    
    
    log_df['input_count'] = log_df['input_count'].astype(int)
    log_df['remaining_count'] = log_df['remaining_count'].astype(int)
    
    return log_df

def apply_parallel(dfGrouped, func, kwargs):
    cpus = cpu_count()
    #cpus = 1
    with Pool(cpus) as p:
        ret_list = p.map(WithExtraArgs(func, **kwargs), [group for name, group in dfGrouped])
    return pd.concat(ret_list)

def generate_predictions(pr, test_data, train_data, 
                         items=None, cut_off=20, algo_name=None, batch_size=100,
                         session_key='SessionId', item_key='ItemId', time_key='Time'): 
    '''
    Evaluates the algorithms wrt. the given metrics. Has no batch evaluation capabilities. 

    Parameters
    --------
    pr : baseline predictor
        A trained instance of a baseline predictor.
    metrics : list
        A list of metric classes providing the proper methods
    test_data : pandas.DataFrame
        Test data. It contains the transactions of the test set.It has one column for session IDs, one for item IDs and one for the timestamp of the events (unix timestamps).
        It must have a header. Column names are arbitrary, but must correspond to the keys you use in this function.
    train_data : pandas.DataFrame
        Training data. Only required for selecting the set of item IDs of the training set.
    items : 1D list or None
        The list of item ID that you want to compare the score of the relevant item to. If None, all items of the training set are used. Default value is None.
    cut-off : int
        Cut-off value (i.e. the length of the recommendation list; N for recall@N and MRR@N). Defauld value is 20.
    batch_size : int
        Number of events bundled into a batch during evaluation. Speeds up evaluation. 
        If it is set high, the memory consumption increases. Default value is 100.
    session_key : string
        Header of the session ID column in the input file (default: 'SessionId')
    item_key : string
        Header of the item ID column in the input file (default: 'ItemId')
    time_key : string
        Header of the timestamp column in the input file (default: 'Time')
    
    Returns
    --------
    out :  list of tuples
        (metric_name, value)
    
    '''
    
    actions = len(test_data)
    sessions = len(test_data[session_key].unique())
    count = 0
    print('START evaluation of ', actions, ' actions in ', sessions, ' sessions')
    
    if algo_name is not "gru":
        print("Running parallel prediction by session groups...")
        items_to_predict = train_data[item_key].unique()

        session_groups = test_data.groupby(session_key)
        # generate predictions in parallel using all cpu cores
        res = apply_parallel(session_groups, predict_function, {"pr": pr, "items_to_predict": items_to_predict})
        res = res.reindex(columns = ["session_id", "input_items", "input_count", "position", "remaining_items", "remaining_count", "predictions"])
        return res
    else:
        print("Running batch prediction for Gru4rec...")
        test_data.sort_values([session_key, time_key], inplace=True)

        offset_sessions = np.zeros(test_data[session_key].nunique()+1, dtype=np.int32)
        offset_sessions[1:] = test_data.groupby(session_key).size().cumsum()
        
        if len(offset_sessions) - 1 < batch_size:
            batch_size = len(offset_sessions) - 1
        
        iters = np.arange(batch_size).astype(np.int32) 

        maxiter = iters.max()    
        start = offset_sessions[iters]
        end = offset_sessions[iters+1]

        in_idx = np.zeros(batch_size, dtype=np.int32)    
        in_sessions = np.zeros(batch_size, dtype = object) 
        np.random.seed(42)
        
        lookup = {}
        
        log_columns = ["session_id", "input_items", "input_count", "position", "remaining_items", "remaining_count", "predictions"]
        log_df = pd.DataFrame(columns = log_columns)
      

        while True:
            valid_mask = iters >= 0
            if valid_mask.sum() == 0:
                break
            
            start_valid = start[valid_mask]        
            minlen = (end[valid_mask]-start_valid).min()
            in_idx[valid_mask] = test_data[item_key].values[start_valid]
            in_sessions[valid_mask] = test_data[session_key].values[start_valid]

            for i in range(minlen-1):
                # expected output
                out_idx = test_data[item_key].values[start_valid+i+1]
                # make prediction
                preds = pr.predict_next_batch(iters, in_idx, None, batch_size)
                
                preds.fillna(0, inplace=True)
                # new in index becomes old out index
                in_idx[valid_mask] = out_idx
                
                valid_sessions = in_sessions[valid_mask]
                i=0
                for part, series in preds.loc[:,valid_mask].iteritems(): 
                    # what is the current session
                    curr_session = valid_sessions[i]
                    # increment item index per session
                    if curr_session in lookup:
                        lookup[curr_session] += 1
                    else:
                        lookup[curr_session] = 0
                    
                    # current session's data
                    cur_session_data = test_data.loc[test_data[session_key] == curr_session]
                    # what were the current inputs
                    history_items = cur_session_data.iloc[:lookup[curr_session] + 1][item_key]
                    # expected outputs
                    to_predict_items = cur_session_data.iloc[lookup[curr_session] + 1:][item_key]
                    # get the top-n predictions
                    preds.sort_values( part, ascending=False, inplace=True )
                    topn_preds = preds[:cut_off]

                    position = "MID"
                    if len(history_items) == 0:
                        position = "FIRST"
                    if len(to_predict_items) == 1:
                        position = "LAST"

                    # use np.array_str as the a new line charachter will be automatically set otherweise
                    log_df = log_df.append({
                        "session_id": curr_session,
                        "input_items":  ','.join(map(str, history_items)),
                        "input_count":  len(history_items),
                        "position": position,
                        "remaining_items":  ','.join(map(str, to_predict_items)),
                        "remaining_count":  len(to_predict_items),
                        "predictions": ','.join(map(str, topn_preds.index.values))
                    }, ignore_index=True)


                    # m.add( preds[part], out_idx[i] )
                    i += 1
            start = start+minlen-1
            mask = np.arange(len(iters))[(valid_mask) & (end-start<=1)]
            for idx in mask:
                maxiter += 1
                if maxiter >= len(offset_sessions)-1:
                    iters[idx] = -1
                else:
                    iters[idx] = maxiter
                    start[idx] = offset_sessions[maxiter]
                    end[idx] = offset_sessions[maxiter+1]
    
        log_df['input_count'] = log_df['input_count'].astype(int)
        log_df['remaining_count'] = log_df['remaining_count'].astype(int)
        return log_df

# Hyperparameter optimization models

In [10]:
store_path = "../../data/recsys17/processed/"
predict_path = "../../data/recsys17/interim/predict/hyperparam/"

valid_models_path = "models/valid"

train  = pd.read_csv(store_path + "valid_train_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]
test = pd.read_csv(store_path + "valid_test_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]

train.columns = ['SessionId', 'ItemId', 'Time']
test.columns = ['SessionId', 'ItemId', 'Time']

for f in listdir(valid_models_path):
    file_path = join(valid_models_path, f)
    if isfile(file_path):
        #if "recsys17" in f:
        if "recsys17" in f and "GRU4Rec" in f:
            print("Loading model: " + file_path)
            loaded_model = pickle.load(open(file_path, 'rb'))
            algo = f.replace("recsys17_","").replace(".model","")
            #res_log = generate_predictions(loaded_model, test, train, algo_name = algo)
            res_log = generate_predictions(loaded_model, test, train, algo_name = "gru")
            res_log.to_csv(predict_path + "test_14d_" + algo + ".csv", sep='\t')
            print("Stored prediction results in: " + predict_path + "test_14d_" + algo + ".csv")


Loading model: models/valid/recsys17_GRU4Rec_lossbpr-max-0.5_layers1000.model
START evaluation of  5958  actions in  2046  sessions
Running batch prediction for Gru4rec...
Stored prediction results in: ../../data/recsys17/interim/predict/hyperparam/test_14d_GRU4Rec_lossbpr-max-0.5_layers1000.csv
Loading model: models/valid/recsys17_GRU4Rec_losstop1-max_layers1000_1000.model
START evaluation of  5958  actions in  2046  sessions
Running batch prediction for Gru4rec...
Stored prediction results in: ../../data/recsys17/interim/predict/hyperparam/test_14d_GRU4Rec_losstop1-max_layers1000_1000.csv
Loading model: models/valid/recsys17_GRU4Rec_lossbpr-max-0.5_layers1000_1000.model
START evaluation of  5958  actions in  2046  sessions
Running batch prediction for Gru4rec...
Stored prediction results in: ../../data/recsys17/interim/predict/hyperparam/test_14d_GRU4Rec_lossbpr-max-0.5_layers1000_1000.csv
Loading model: models/valid/recsys17_GRU4Rec_losstop1-max_layers100_100.model
START evaluation 

# Evaluation

In [4]:
def predict_algo(algo):
    load_path = "../../data/recsys17/processed/"

    train  = pd.read_csv(load_path + "train_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]
    test = pd.read_csv(load_path + "test_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]

    train.columns = ['SessionId', 'ItemId', 'Time']
    test.columns = ['SessionId', 'ItemId', 'Time']
    
    filename = "models/recsys17_"  + algo + ".model"
    print("Loading model: " + filename)
    loaded_model = pickle.load(open(filename, 'rb'))
    
    res_log = generate_predictions(loaded_model, test, train, algo_name = algo)
    
    store_path = "../../data/recsys17/interim/predict/base/"
    res_log.to_csv(store_path + "test_14d_" + algo + ".csv", sep='\t')
    print("Stored prediction results in: " + store_path + "test_14d_" + algo + ".csv")
    pass

In [5]:
predict_algo("pop")
predict_algo("iknn")
predict_algo("bpr")
predict_algo("cknn")
predict_algo("scknn")
predict_algo("vcknn")
predict_algo("gru")

Loading model: models/recsys17_pop.model
START evaluation of  8498  actions in  3610  sessions
Running parallel prediction by session groups...
Stored prediction results in: ../../data/recsys17/interim/predict/base/test_14d_pop.csv
Loading model: models/recsys17_iknn.model
START evaluation of  8498  actions in  3610  sessions
Running parallel prediction by session groups...
Stored prediction results in: ../../data/recsys17/interim/predict/base/test_14d_iknn.csv
Loading model: models/recsys17_bpr.model
START evaluation of  8498  actions in  3610  sessions
Running parallel prediction by session groups...
Stored prediction results in: ../../data/recsys17/interim/predict/base/test_14d_bpr.csv
Loading model: models/recsys17_cknn.model
START evaluation of  8498  actions in  3610  sessions
Running parallel prediction by session groups...
Stored prediction results in: ../../data/recsys17/interim/predict/base/test_14d_cknn.csv
Loading model: models/recsys17_scknn.model
START evaluation of  8498

In [10]:
# ---------------------------
# PREDICT
# ---------------------------
# ---------------------------

algo = "gru" # pop, spop, iknn, cknn, bpr, gru

load_path = "../data/recsys17/processed/"

train  = pd.read_csv(load_path + "train_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]
test = pd.read_csv(load_path + "test_14d.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]

train.columns = ['SessionId', 'ItemId', 'Time']
test.columns = ['SessionId', 'ItemId', 'Time']

filename = "../data/recsys17/interim/models/"  + algo + ".model"

print("Loading model: " + filename)
loaded_model = pickle.load(open(filename, 'rb'))

res_log = generate_predictions(loaded_model, test, train, algo_name = algo)

store_path = "../data/recsys17/interim/predict/base_"
res_log.to_csv(store_path + "test_14d_" + algo + ".csv", sep='\t')
print("Stored prediction results in: " + store_path + "test_14d_" + algo + ".csv")


Loading model: ../data/recsys17/interim/models/gru.model
START evaluation of  8498  actions in  3610  sessions
Running batch prediction for Gru4rec...
Stored prediction results in: ../data/recsys17/interim/predict/base_test_14d_gru.csv


In [7]:
# ---------------------------
# PREDICT
# ---------------------------
# ---------------------------
store_path = "../data/recsys17/"

train  = pd.read_csv(store_path + "train_d14.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]
test = pd.read_csv(store_path + "test_d14.csv", sep='\t', dtype={'item_id':np.int64})[['session_id','item_id','created_at']]

train.columns = ['SessionId', 'ItemId', 'Time']
test.columns = ['SessionId', 'ItemId', 'Time']

algo = "gru"
filename = "Models/recsys17_"  + algo + ".model"

print("Loading model: " + filename)
loaded_model = pickle.load(open(filename, 'rb'))

res_log = generate_predictions(loaded_model, test, train, algo_name = algo)

store_path = "../results/recsys17/base/"
res_log.to_csv(store_path + "test_d14_" + algo + ".csv", sep='\t')
print("Stored prediction results in: " + store_path + "test_d14_" + algo + ".csv")


Loading model: Models/no_repeat_recsys17_gru.model
START evaluation of  2594  actions in  997  sessions
Running batch prediction for Gru4rec...
Stored prediction results in: ../results/recsys17/base/no_repeat_test_d14_gru.csv
